# Machine Learning HW.3 

# CNN-Classification

Dataset: https://www.kaggle.com/c/ml2020spring-hw3/data

In [49]:
import os
import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

In [55]:
def read_photo(path,label = False):
    path_list = os.listdir(path)
    x = np.zeros([len(path_list), 128, 128, 3])
    y = np.zeros([len(path_list), 1])
    for i,f in enumerate(path_list):
        img = cv2.imread(os.path.join(path, f))
        x[i,:,:] = cv2.resize(img, (128, 128))
        if label:
            y[i,:] = int(f.split('_')[0])
    if label:
        return x, y
    else:
        return x

In [56]:
path = '/Users/Juyi/Desktop/Code/Data/food-11'
train_data, train_label = read_photo(os.path.join(path, 'training'), True)
val_data, val_label = read_photo(os.path.join(path, 'validation'), True)
testing_data = read_photo(os.path.join(path, 'testing'))

In [57]:
print(train_data.shape,train_label.shape)
print(val_data.shape,val_label.shape)
print(testing_data.shape)

(9866, 128, 128, 3) (9866, 1)
(3430, 128, 128, 3) (3430, 1)
(3110, 128, 128, 3)
